In [1]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#         .appName("FlightData")\
#         .getOrCreate()

# spark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("FlightData") \
        .getOrCreate()

spark

In [2]:
# df = spark.read.option("header", True).option("inferSchema", True).csv("Flight_Schedule.csv")
# df.show(5)
# df.printSchema()

df = spark.read.option("header", True).option("inferSchema", True).csv("Flight_Schedule.csv")
df.show(5)
df.printSchema()

+------------+-------+------+-----------+--------------------+----------------------+--------------------+----------+----------+
|flightNumber|airline|origin|destination|           dayOfWeek|scheduledDepartureTime|scheduledArrivalTime| validFrom|   validTo|
+------------+-------+------+-----------+--------------------+----------------------+--------------------+----------+----------+
|         425|  GoAir| Delhi|  Hyderabad|Sunday,Monday,Tue...|   2025-04-15 05:45:00|                NULL|28-10-2018|30-03-2019|
|         423|  GoAir| Delhi|  Hyderabad|            Saturday|   2025-04-15 07:30:00|                NULL|28-10-2018|28-10-2018|
|         423|  GoAir| Delhi|  Hyderabad|              Friday|   2025-04-15 07:30:00|                NULL|03-11-2018|01-12-2018|
|         423|  GoAir| Delhi|  Hyderabad|              Friday|   2025-04-15 07:30:00|                NULL|02-02-2019|30-03-2019|
|         423|  GoAir| Delhi|  Hyderabad|Sunday,Monday,Tue...|   2025-04-15 07:30:00|            

In [4]:
# from pyspark.sql.functions import col, to_date, year, dayofmonth, month
# splitDate = df.withColumn("validFromDate", to_date(col("ValidFrom"), "dd-MM-yyyy")) \
#                 .withColumn("validFromYear", year(col("validFromDate"))) \
#                 .withColumn("validFromMonth", month(col("validFromDate"))) \
#                 .withColumn("validFromDay", dayofmonth(col("validFromDate"))) \
#                 .withColumn("validToDate", to_date(col("ValidTo"), "dd-MM-yyyy")) \
#                 .withColumn("validToYear", year(col("validToDate"))) \
#                 .withColumn("validToMonth", month(col("validToDate"))) \
#                 .withColumn("validToDay", dayofmonth(col("validToDate")))

# splitDate.select("flightNumber","airline","origin","destination","validFromDate", "validToDate", "validFromYear","validToYear","validFromMonth","validToMonth") \
#             .distinct().show()

# from pyspark.sql.types import IntegerType

# df = df.withColumn("flightNumber", df["flightNumber"].cast(IntegerType()))
# df.printSchema()

from pyspark.sql.functions import col, to_date, year, dayofmonth, month

splitDate = df.withColumn("validFromDate", to_date(col("ValidFrom"), "dd-MM-yyyy")) \
                .withColumn("validFromYear", year(col("validFromDate"))) \
                .withColumn("validFromMonth", month(col("validFromDate"))) \
                .withColumn("validFromDay", dayofmonth(col("validFromDate"))) \
                .withColumn("validToDate", to_date(col("ValidTo"), "dd-MM-yyyy")) \
                .withColumn("validToYear", year(col("validToDate"))) \
                .withColumn("validToMonth", month(col("validToDate"))) \
                .withColumn("validToDay", dayofmonth(col("validToDate")))

splitDate.select("flightNumber","airline","origin","destination","validFromDate", "validToDate", "validFromYear","validToYear","validFromMonth","validToMonth") \
             .distinct().show()


+------------+-------------+-----------+-----------+-------------+-----------+-------------+-----------+--------------+------------+
|flightNumber|      airline|     origin|destination|validFromDate|validToDate|validFromYear|validToYear|validFromMonth|validToMonth|
+------------+-------------+-----------+-----------+-------------+-----------+-------------+-----------+--------------+------------+
|         304|        GoAir| Port Blair|    Chennai|   2018-10-28| 2019-03-30|         2018|       2019|            10|           3|
|         407|        GoAir|Bhubaneswar|    Kolkata|   2018-10-28| 2019-03-30|         2018|       2019|            10|           3|
|         405|    Air India|  Khajuraho|   Varanasi|   2018-10-29| 2019-03-30|         2018|       2019|            10|           3|
|         441|    Air India| Aurangabad|     Mumbai|   2018-10-28| 2019-03-30|         2018|       2019|            10|           3|
|         569|    Air India|    Chennai|     Mumbai|   2018-10-28| 20

In [5]:
# from pyspark.sql.functions import col
# filtered_df = df1.filter((col("origin") == "Delhi") &
#                         (col("destination") == "Hyderabad") &
#                         (col("dayOfWeek").contains("Saturday")))

# filtered_df.show()

# from pyspark.sql.types import IntegerType, DoubleType
# splitDate = splitDate.withColumn("flightNumber", df["flightNumber"].cast(IntegerType()))
# splitDate.createOrReplaceTempView("flights")

# sqlAirline = """
#         SELECT flightNumber, airline, COUNT(airline) as totalFlight
#         FROM flights
#         WHERE airline IS NOT NULL AND flightNumber IS NOT NULL
#         GROUP BY flightNumber,airline
#         ORDER BY flightNumber ASC"""
# flightData = spark.sql(sqlAirline)
# flightData.show()
# from pyspark.sql.types import IntegerType, StringType
# splitDate= splitDate.withColumn("flightNumber", splitDate["flightNumber"].cast(IntegerType()))
# splitDate.createOrReplaceTempView("flights")

# flightSQL = """
#         SELECT DISTINCT flightNumber, airline, origin,destination,COUNT(airline) as TotalFlight, validFromYear, validToYear
#         FROM flights
#         WHERE airline IS NOT NULL AND flightNumber IS NOT NULL
#         GROUP BY flightNumber, airline, origin,destination, validFromYear, validToYear
#         ORDER BY flightNumber DESC"""
# flightData = spark.sql(flightSQL)
# flightData.show()

df_cleaned = splitDate.na.drop()
print("DataFrame after removing nulls:")
df_cleaned.show(5)

print(f"Original row count: {splitDate.count()}")
print(f"Cleaned row count: {df_cleaned.count()}")


DataFrame after removing nulls:
+------------+-------+---------+-----------+--------------------+----------------------+--------------------+----------+----------+-------------+-------------+--------------+------------+-----------+-----------+------------+----------+
|flightNumber|airline|   origin|destination|           dayOfWeek|scheduledDepartureTime|scheduledArrivalTime| validFrom|   validTo|validFromDate|validFromYear|validFromMonth|validFromDay|validToDate|validToYear|validToMonth|validToDay|
+------------+-------+---------+-----------+--------------------+----------------------+--------------------+----------+----------+-------------+-------------+--------------+------------+-----------+-----------+------------+----------+
|         559|  GoAir|  Lucknow|  Hyderabad|Sunday,Tuesday,We...|   2025-04-15 15:45:00| 2025-04-15 17:45:00|28-10-2018|30-03-2019|   2018-10-28|         2018|            10|          28| 2019-03-30|       2019|           3|        30|
|         580|  GoAir|  

In [6]:
# flight_count = filtered_df.count()
# print(f"Number of Delhi-Hyderabad Saturday flights: {flight_count}")
# flightData20 = flightData.filter(flightData["totalFlight"] > 30)
# flightData10 = flightData20.filter(flightData20["totalFlight"] > 10)

# flightData20.show()
# flightData10.show()

# flightData.write.partitionBy("airline").parquet("output")
df_cleaned.createOrReplaceTempView("flights")
flightSQL = """
    SELECT DISTINCT flightNumber, airline, origin,destination,COUNT(airline) as TotalFlight, validFromYear, validToYear
          FROM flights
          WHERE airline IS NOT NULL AND flightNumber IS NOT NULL
          GROUP BY flightNumber, airline, origin,destination, validFromYear, validToYear
          ORDER BY flightNumber DESC"""

flightData = spark.sql(flightSQL)
flightData.show()

+------------+-----------+------------------+------------------+-----------+-------------+-----------+
|flightNumber|    airline|            origin|       destination|TotalFlight|validFromYear|validToYear|
+------------+-----------+------------------+------------------+-----------+-------------+-----------+
|        9820|Jet Airways|            Imphal|           Kolkata|          1|         2018|       2019|
|        9819|Jet Airways|           Kolkata|            Imphal|          1|         2018|       2019|
|        9818|Jet Airways|          Srinagar|             Delhi|          1|         2018|       2019|
|        9817|Jet Airways|             Delhi|          Srinagar|          1|         2018|       2019|
|        8964|   SpiceJet|Thiruvananthapuram|             Delhi|          1|         2019|       2020|
|        8964|   SpiceJet|Thiruvananthapuram|             Delhi|          1|         2020|       2020|
|        8963|   SpiceJet|             Delhi|Thiruvananthapuram|         

In [12]:
# jetAirWaysDF = spark.read.parquet("output/airline=AirAsia India/part-00000-2af11fb5-2af2-4daa-a534-2d12c4d68883.c000.snappy.parquet")
# jetAirWaysDF.show()
# flightData.write.partitionBy("airline").parquet("output")

airlineGoAir = spark.read.parquet("output/airline=GoAir")
airlineGoAir.show()

airlineGoAir.createOrReplaceTempView("goAir")
goAirData = spark.sql(""" 
                    SELECT * FROM goAir where TotalFlight > 2""")
goAirData.show()

+------------+-------------+-------------+-----------+-------------+-----------+
|flightNumber|       origin|  destination|TotalFlight|validFromYear|validToYear|
+------------+-------------+-------------+-----------+-------------+-----------+
|        2601|       Mumbai|       Nagpur|          1|         2019|       2020|
|        2610|        Kochi|Visakhapatnam|          1|         2019|       2019|
|        2606|          Goa|       Mumbai|          1|         2019|       2020|
|        2601|       Mumbai|       Nagpur|          1|         2020|       2021|
|        2603|       Mumbai|       Indore|          1|         2020|       2020|
|        2606|          Goa|       Mumbai|          1|         2020|       2020|
|        2604|       Indore|       Mumbai|          1|         2020|       2020|
|        2607|       Mumbai|       Jaipur|          1|         2020|       2020|
|        2608|       Jaipur|       Mumbai|          1|         2019|       2020|
|        2602|       Nagpur|

In [14]:
# jetAirWaysDF.createOrReplaceTempView("airasia")

# airasiasql = """
#     SELECT * from airasia where TotalFlight > 10 """

# airasia1 = spark.sql(airasiasql)
# airasia1.show()
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def flightperformance(payload):
    if payload > 3:
        return "Good"
    else:
        return "Bad"
regUDF = udf(flightperformance, StringType())
goAirData = goAirData.withColumn("flighPerformance", regUDF(goAirData["TotalFlight"]))
goAirData.show()


+------------+----------+-----------+-----------+-------------+-----------+----------------+
|flightNumber|    origin|destination|TotalFlight|validFromYear|validToYear|flighPerformance|
+------------+----------+-----------+-----------+-------------+-----------+----------------+
|         803|Port Blair|  Bengaluru|          3|         2019|       2019|             Bad|
|         802| Bengaluru| Port Blair|          5|         2019|       2019|            Good|
|         374| Bengaluru|     Ranchi|          4|         2020|       2020|            Good|
|         375|    Ranchi|  Bengaluru|          4|         2020|       2020|            Good|
+------------+----------+-----------+-----------+-------------+-----------+----------------+



In [21]:
# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType

# def flightStatus(payload):
#     if payload >= 30:
#         return "Good"
#     else:
#         return "Bad"
# registerUDF = udf(flightStatus,StringType())
# dfStatus = flightData.withColumn("flightCondition", registerUDF(flightData["totalFlight"]))
# dfStatus.show()
airlineGoAir.write.bucketBy(5,"origin") \
            .sortBy("origin") \
            .mode("overwrite") \
            .saveAsTable("flight_bucketed")
bucketed_df = spark.table("flight_bucketed")
bucketed_df.show(5)

bucketed_df.printSchema()

+------------+--------+-----------+-----------+-------------+-----------+
|flightNumber|  origin|destination|TotalFlight|validFromYear|validToYear|
+------------+--------+-----------+-----------+-------------+-----------+
|         982|Bagdogra|    Chennai|          1|         2018|       2019|
|         851|Bagdogra|      Delhi|          1|         2019|       2020|
|         851|Bagdogra|      Delhi|          1|         2019|       2019|
|         758|Bagdogra|      Delhi|          1|         2020|       2020|
|         758|Bagdogra|      Delhi|          1|         2019|       2019|
+------------+--------+-----------+-----------+-------------+-----------+
only showing top 5 rows

root
 |-- flightNumber: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)
 |-- TotalFlight: long (nullable = true)
 |-- validFromYear: integer (nullable = true)
 |-- validToYear: integer (nullable = true)



In [24]:
# filtered_df.printSchema()
# dfStatus.createOrReplaceTempView("condition")
# conditionSql = """
#     SELECT flightNumber, MAX(totalFlight) as MaxVal FROM condition
#     GROUP BY flightNumber
#     ORDER BY MaxVal DESC
#     """
# conditionTest = spark.sql(conditionSql)
# conditionTest.show()
# spark.sql("""
#     CREATE EXTERNAL TABLE flight_external (
#         flightNumber Integer,
#         airline STRING,
#         destination STRING,
#         TotalFlight long ,
#         validFromYear Integer,
#         validToYear Integer
#     )
#     PARTITIONED BY (origin STRING)
#     STORED AS PARQUET
#     LOCATION 'spark-warehouse/flight_bucketed'
# """)
# spark.sql("MSCK REPAIR TABLE flight_external")

# spark.sql("""
#     SELECT flightNumber, airline, destination, validFrom
#     FROM flight_external
#     WHERE origin = 'Delhi'
# """).show(5)
from pyspark.sql.functions import array

# Assuming your DataFrame is named 'df'
goAirData = goAirData.withColumn(
    "combined_array",
    array("TotalFlight", "validFromYear", "validToYear", "flighPerformance")
)

# Show the result
goAirData.show()

+------------+----------+-----------+-----------+-------------+-----------+----------------+--------------------+
|flightNumber|    origin|destination|TotalFlight|validFromYear|validToYear|flighPerformance|      combined_array|
+------------+----------+-----------+-----------+-------------+-----------+----------------+--------------------+
|         803|Port Blair|  Bengaluru|          3|         2019|       2019|             Bad|[3, 2019, 2019, Bad]|
|         802| Bengaluru| Port Blair|          5|         2019|       2019|            Good|[5, 2019, 2019, G...|
|         374| Bengaluru|     Ranchi|          4|         2020|       2020|            Good|[4, 2020, 2020, G...|
|         375|    Ranchi|  Bengaluru|          4|         2020|       2020|            Good|[4, 2020, 2020, G...|
+------------+----------+-----------+-----------+-------------+-----------+----------------+--------------------+



In [33]:
# filtered_df.select("origin","destination", "dayOfWeek").distinct().show()
from pyspark.sql.types import IntegerType
def addData(payload):
    if payload > 30:
        return payload + 1
    else:
        return payload + 10
registerAddData = udf(addData, IntegerType())
dfAddData =  conditionTest.withColumn("AddCondition", registerAddData(conditionTest["MaxVal"]))
dfAddData.show()

+------------+------+------------+
|flightNumber|MaxVal|AddCondition|
+------------+------+------------+
|         559|    90|          91|
|         612|    86|          87|
|        2197|    71|          72|
|         611|    71|          72|
|         238|    66|          67|
|         748|    62|          63|
|         287|    62|          63|
|        1451|    56|          57|
|         496|    50|          51|
|        1427|    50|          51|
|         634|    46|          47|
|         406|    45|          46|
|        2148|    45|          46|
|         237|    45|          46|
|         802|    45|          46|
|         587|    43|          44|
|        6325|    43|          44|
|         741|    43|          44|
|        1426|    41|          42|
|         477|    41|          42|
+------------+------+------------+
only showing top 20 rows



In [35]:
# from pyspark.sql.functions import col, when, split, array_contains

# days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# df_split = df1.withColumn("dayOfWeek_array", split(col("dayOfWeek"), ","))

# for day in days:
#     df_split = df_split.withColumn(
#         day,
#         when(array_contains(col("dayOfWeek_array"), day), 1).otherwise(0)
#     )

# df_result = df_split.drop("dayOfWeek_array")

# # Show the result
# df_result.show()

# # Print schema to verify new columns
# df_result.printSchema()
dfAddData.createOrReplaceTempView("dataview")
maxSql = """
    SELECT * FROM dataview where MaxVal > 10
    ORDER BY MaxVal ASC """
maxsql1 = spark.sql(maxSql)
maxsql1.show()

+------------+------+------------+
|flightNumber|MaxVal|AddCondition|
+------------+------+------------+
|        3071|    11|          21|
|        2554|    11|          21|
|        2031|    11|          21|
|        7993|    11|          21|
|         727|    11|          21|
|        3220|    11|          21|
|         686|    11|          21|
|        7206|    11|          21|
|        7265|    11|          21|
|         321|    11|          21|
|        7989|    11|          21|
|        2871|    11|          21|
|         367|    11|          21|
|        1516|    11|          21|
|         191|    11|          21|
|        8144|    11|          21|
|        1721|    11|          21|
|         760|    11|          21|
|         961|    11|          21|
|         328|    11|          21|
+------------+------+------------+
only showing top 20 rows



In [13]:
df_result.filter(
    (col("Monday") == 1) &
    (col("Tuesday") == 1) &
    (col("Wednesday") == 1) &
    (col("Thursday") == 1) &
    (col("Friday") == 1) &
    (col("Saturday") == 1) &
    (col("Sunday") == 1)
).select("flightNumber","airline","scheduledDepartureTime","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday").distinct().show()

+------------+-----------+----------------------+------+-------+---------+--------+------+--------+------+
|flightNumber|    airline|scheduledDepartureTime|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday|
+------------+-----------+----------------------+------+-------+---------+--------+------+--------+------+
|         635|  Air India|   2025-04-12 09:15:00|     1|      1|        1|       1|     1|       1|     1|
|         849|  Air India|   2025-04-12 16:10:00|     1|      1|        1|       1|     1|       1|     1|
|         148|Jet Airways|   2025-04-12 19:50:00|     1|      1|        1|       1|     1|       1|     1|
|         986| TestIndigo|   2025-04-12 11:40:00|     1|      1|        1|       1|     1|       1|     1|
|         473| TestIndigo|   2025-04-12 21:10:00|     1|      1|        1|       1|     1|       1|     1|
|        7106| TestIndigo|   2025-04-12 08:10:00|     1|      1|        1|       1|     1|       1|     1|
|        7989| TestIndigo|   2025-04-

In [23]:
from pyspark.sql.functions import to_date,month, year

df_with_month = df1.withColumn("validFrom", to_date(col("scheduledDepartureTime"))) \
                    .withColumn("year", year(col("date"))) \
                    .withColumn("month", month(col("date")))

df_with_month.filter(col("month") != 4 ).select("date", "year","month").show(10)


+----+----+-----+
|date|year|month|
+----+----+-----+
+----+----+-----+



In [19]:
df_grouped = df_with_month.groupBy("airline","year", "month") \
                            .agg({"FlightNumber":"count"}) \
                            .withColumnRenamed("count(flightNumber)", "flight_count") \
                            .orderBy("airline", "year","month")
df_grouped.show()

+-------------+----+-----+------------+
|      airline|year|month|flight_count|
+-------------+----+-----+------------+
|         NULL|NULL| NULL|         595|
|         NULL|2025|    4|        1291|
|    Air India|NULL| NULL|        1108|
|    Air India|2025|    4|        2379|
|AirAsia India|NULL| NULL|         664|
|AirAsia India|2025|    4|        1699|
|        GoAir|NULL| NULL|        1380|
|        GoAir|2025|    4|        2449|
|  Jet Airways|NULL| NULL|         325|
|  Jet Airways|2025|    4|         903|
|      Jetlite|NULL| NULL|          23|
|      Jetlite|2025|    4|          82|
|     SpiceJet|NULL| NULL|        1262|
|     SpiceJet|2025|    4|        3262|
|   TestIndigo|NULL| NULL|        4458|
|   TestIndigo|2025|    4|       10137|
|       TruJet|NULL| NULL|         173|
|       TruJet|2025|    4|         382|
|      Vistara|NULL| NULL|         590|
|      Vistara|2025|    4|        1163|
+-------------+----+-----+------------+



In [3]:
df = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("Flight_Schedule.csv")
df.show(10)

+------------+-------+-------+-----------+--------------------+----------------------+--------------------+----------+----------+
|flightNumber|airline| origin|destination|           dayOfWeek|scheduledDepartureTime|scheduledArrivalTime| validFrom|   validTo|
+------------+-------+-------+-----------+--------------------+----------------------+--------------------+----------+----------+
|         425|  GoAir|  Delhi|  Hyderabad|Sunday,Monday,Tue...|   2025-04-12 05:45:00|                NULL|28-10-2018|30-03-2019|
|         423|  GoAir|  Delhi|  Hyderabad|            Saturday|   2025-04-12 07:30:00|                NULL|28-10-2018|28-10-2018|
|         423|  GoAir|  Delhi|  Hyderabad|              Friday|   2025-04-12 07:30:00|                NULL|03-11-2018|01-12-2018|
|         423|  GoAir|  Delhi|  Hyderabad|              Friday|   2025-04-12 07:30:00|                NULL|02-02-2019|30-03-2019|
|         423|  GoAir|  Delhi|  Hyderabad|Sunday,Monday,Tue...|   2025-04-12 07:30:00|    

In [7]:
from pyspark.sql.functions import col, to_date, year, month, dayofmonth

splitDate = df.withColumn("validFrom_date", to_date(col("validFrom"), "dd-MM-yyyy")) \
                        .withColumn("validFrom_year", year(col("validFrom_date"))) \
                        .withColumn("validFrom_month", month(col("validFrom_date"))) \
                        .withColumn("validFrom_day", dayofmonth(col("validFrom_date"))) \
                        .withColumn("validTo_date", to_date(col("validTo"), "dd-MM-yyyy")) \
                        .withColumn("validTo_year", year(col("validTo_date"))) \
                        .withColumn("validTo_month", month(col("validTo_date"))) \
                        .withColumn("validTo_day", dayofmonth(col("validTo_date")))
splitDate.select(
    "airline", "flightNumber","validFrom", "validFrom_year", "validFrom_month", "validFrom_day",
    "validTo", "validTo_year", "validTo_month", "validTo_day"
).distinct().show()
splitDate.printSchema()

+-------------+------------+----------+--------------+---------------+-------------+----------+------------+-------------+-----------+
|      airline|flightNumber| validFrom|validFrom_year|validFrom_month|validFrom_day|   validTo|validTo_year|validTo_month|validTo_day|
+-------------+------------+----------+--------------+---------------+-------------+----------+------------+-------------+-----------+
|        GoAir|         468|01-12-2018|          2018|             12|            1|30-03-2019|        2019|            3|         30|
|        GoAir|         418|29-10-2018|          2018|             10|           29|30-03-2019|        2019|            3|         30|
|    Air India|         853|28-10-2018|          2018|             10|           28|30-03-2019|        2019|            3|         30|
|    Air India|         421|30-10-2018|          2018|             10|           30|22-11-2018|        2018|           11|         22|
|AirAsia India|         783|28-10-2018|          2018| 

In [12]:
airlineDataByYear = splitDate.write \
                    .partitionBy("airline", "validFrom_year","validFrom_month") \
                    .mode("overwrite") \
                    .parquet("output/flight_schedule_partitioned")

In [15]:
df_verify = spark.read.parquet("output/flight_schedule_partitioned/airline=GoAir/validFrom_year=2018/validFrom_month=10")
df_verify.show(5)

+------------+-------+-----------+--------------------+----------------------+--------------------+----------+----------+--------------+-------------+------------+------------+-------------+-----------+
|flightNumber| origin|destination|           dayOfWeek|scheduledDepartureTime|scheduledArrivalTime| validFrom|   validTo|validFrom_date|validFrom_day|validTo_date|validTo_year|validTo_month|validTo_day|
+------------+-------+-----------+--------------------+----------------------+--------------------+----------+----------+--------------+-------------+------------+------------+-------------+-----------+
|         425|  Delhi|  Hyderabad|Sunday,Monday,Tue...|   2025-04-12 05:45:00|                NULL|28-10-2018|30-03-2019|    2018-10-28|           28|  2019-03-30|        2019|            3|         30|
|         423|  Delhi|  Hyderabad|            Saturday|   2025-04-12 07:30:00|                NULL|28-10-2018|28-10-2018|    2018-10-28|           28|  2018-10-28|        2018|           1